# Poschl-Teller Potential

The Poschl-Teller potential is given by: \
\
$ V(x) = \dfrac{1}{2}\,\text{sech}^2(x) = \dfrac{2}{(e^{-x}+e^x)^2} $

# Schrödinger-like equation manipulation to find form compatible with AIM

We begin our analysis with the Schrödinger equation: \
\
$ \dfrac{d^2\psi}{dx^2} + \left( \omega^2 - \dfrac{1}{2}\text{sech}^2(x) \right)\psi = 0 $

It is convenient to write it in terms of a bounded variable. For this, we perform the change of variables $y = \tanh(x)$.

In [79]:
from sympy import *

x,y = symbols("x y", real=True)
w = symbols("\omega")

psi = Function("\psi")(x)
psi_y = Function("\psi")(y)
phi = Function("\phi")(y)

sch_eq = Eq( diff(diff(psi,x),x) + (w**2-S(1)/2*sech(x)**2)*psi, 0 )
display(sch_eq)

Eq((\omega**2 - sech(x)**2/2)*\psi(x) + Derivative(\psi(x), (x, 2)), 0)

In [80]:
#Function to change variables (https://stackoverflow.com/questions/57840957/differential-equation-change-of-variables-with-sympy)
def variable_change(ODE,dependent_var, 
                    independent_var,
                    new_dependent_var = None, 
                    new_independent_var= None, 
                    dependent_var_relation = None,
                    independent_var_relation = None,
                    order = 2):

    if new_dependent_var == None:
        new_dependent_var = dependent_var
    if new_independent_var == None:
        new_independent_var = independent_var

    # dependent variable change

    if new_independent_var != independent_var:

        for i in range(order, -1, -1):

            # remplace derivate
            a = diff(dependent_var , independent_var, i )
            ξ = Function("ξ")(independent_var)

            b = diff( dependent_var.subs(independent_var, ξ),  independent_var  ,i)

            rel = solve(independent_var_relation, new_independent_var)[0]

            for j in range(order, 0, -1):
                b = b.subs( diff(ξ,independent_var,j), diff(rel,independent_var,j))

            b = b.subs(ξ, new_independent_var)

            rel = solve(independent_var_relation, independent_var)[0]
            b = b.subs(independent_var, rel)

            ODE =   ODE.subs(a,b)

        ODE = ODE.subs(independent_var, rel)

    # change of variables of indpendent variable

    if new_dependent_var != dependent_var:

        ODE = (ODE.subs(dependent_var.subs(independent_var,new_independent_var) , (solve(dependent_var_relation, dependent_var)[0])))
        ODE = ODE.doit().expand()

    return ODE.simplify()

In [81]:
sch_eq_y = variable_change(
    ODE=sch_eq,
    independent_var=x,
    new_independent_var=y,
    independent_var_relation=Eq(x,atanh(y)),
    dependent_var=psi,
    new_dependent_var=psi,
    dependent_var_relation=None,
    order=2
)

display(sch_eq_y)

Eq(2*y*(y**2 - 1)*Derivative(\psi(y), y) + (y**2 - 1)**2*Derivative(\psi(y), (y, 2)) + (2*\omega**2 + y**2 - 1)*\psi(y)/2, 0)

In [90]:
sch_eq_y_lhs = sch_eq_y.lhs*(y**2-1)**(-2)
sch_eq_y_lhs.doit()
sch_eq_y_lhs.expand()

\omega**2*\psi(y)/(y**4 - 2*y**2 + 1) + y**4*Derivative(\psi(y), (y, 2))/(y**4 - 2*y**2 + 1) + 2*y**3*Derivative(\psi(y), y)/(y**4 - 2*y**2 + 1) + y**2*\psi(y)/(2*y**4 - 4*y**2 + 2) - 2*y**2*Derivative(\psi(y), (y, 2))/(y**4 - 2*y**2 + 1) - 2*y*Derivative(\psi(y), y)/(y**4 - 2*y**2 + 1) - \psi(y)/(2*y**4 - 4*y**2 + 2) + Derivative(\psi(y), (y, 2))/(y**4 - 2*y**2 + 1)

In [98]:
A,B,C = symbols("A B C") #A,B,C represent second, first and zero-th order derivatives of phi
sch_eq_y_lhs = sch_eq_y_lhs.subs( diff(diff(psi_y,y),y), A ).subs( diff(psi_y,y), B ).subs(psi_y, C)
display(sch_eq_y_lhs)

(A*(y**2 - 1)**2 + 2*B*y*(y**2 - 1) + C*(2*\omega**2 + y**2 - 1)/2)/(y**2 - 1)**2

Now, we substitute the function $\psi$ to include the boundary conditions

In [ ]:
psi_new = (1-y)**(-I*w/2)*(1+y)**(-I*w/2)*phi
display(psi_new)

In [ ]:
sch_eq_y = sch_eq_y.subs(psi_y,psi_new).doit()
display(sch_eq_y)

Schrödinger-like equation after change of coordinates

$ \dfrac{d^2\psi}{dx^2} + \left( \omega^2 - \dfrac{1}{2}\text{sech}^2(x) \right)\psi = 0 \implies  \dfrac{d^2\phi}{dy^2} = \dfrac{2y(1-iw)}{1-y^2}\dfrac{d\phi}{dy} + \dfrac{1-2iw-2w^2}{2(1-y^2)}\phi $

$ \text{where } y = \text{tanh}(x) $

### QNM Boundary Conditions

$ \phi = (1-y)^{-iw/2}(1+y)^{-iw/2} $

$ \text{which fulfills } y \rightarrow 1 \implies \psi \sim e^{\mp iwx} \sim (1-y)^{\pm iw/2} \implies \psi \sim (1-y)^{-iw/2} $

$ \text{and } y \rightarrow -1 \implies \phi \sim e^{\pm iwx} \sim (1+y)^{\pm iw/2} \implies \psi \sim (1+y)^{-iw/2} $

### Parameters

$ \lambda_0 = \dfrac{2y(1-iw)}{1-y^2} \qquad s_0 = \dfrac{1-2iw-2w^2}{2(1-y^2)} $

### AIM Method
$ \lambda_n = \lambda'_{n-1} + s_{n-1} + \lambda_0\lambda_{n-1} \qquad s_n = s'_{n-1} + s_0\lambda_{n-1} $


Quantization condition: $ \delta_n = s_n\lambda_{n-1} - s_{n-1}\lambda_n $

# AIM (simple) algorithm

Import necessary libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sympy import *

N = 1000

y = symbols("y", real=True)
w = symbols("\omega")

l = np.empty(N,dtype=object)
lp = np.empty(N,dtype=object)
s = np.empty(N,dtype=object)
sp = np.empty(N,dtype=object)

d = np.empty(N,dtype=object)

l[0] = (2*y*(1-I*w))/(1-y**2)
s[0] = (1-2*I*w-2*w**2)/(2*(1-y**2))

First, find the maximum/minimum value of the potential to evaluate at

In [ ]:
x = symbols("x", real=True)
V = 2/( ( E**(-x) + E**x )**2 )
p1 = plot(V)
p1

In [ ]:
Vp = diff(V,x)
critp = solve(Vp,x)
critp

Next, compute first derivatives of lambda and s parameters

In [ ]:
lp[0] = diff(l[0],y)
sp[0] = diff(s[0],y)

In [ ]:
lp[0]

In [ ]:
sp[0]

Calculate n-th (1st) values of lambda and s parameter

In [ ]:
l[1] = lp[0]+s[0]+l[0]*l[0]
s[1] = sp[0] + s[0]*l[0]

In [ ]:
series(l[1],y,x0=0,n=5)

In [ ]:
series(s[1],y,x0=0,n=5)

Evaluate the quantization condition $\delta_n$ at point $y = 0$ obtaining polynomial in $\omega$

In [ ]:
d[1] = s[1]*l[0] - s[0]*l[1]
#d[1].subs(y,0)
p = simplify(d[1].subs(y,0))
d[1]


In [ ]:
p

Solve the polynomial for the solution of $\omega$, taking only those with negative imaginary part and positive real part

In [ ]:
sols = solve(p)
print("All solutions: " + str(sols))
print("Filtered solutions:")
for i in range(len(sols)):
    if re(sols[i]) > 0 and  im(sols[i]) < 0:
        print("w_" + str(i+1) + " = " + str(sols[i]))

### Second Iteration

In [ ]:
lp[1] = diff(l[1],y)
sp[1] = diff(s[1],y)
l[2] = lp[1] + s[1] + l[0]*l[1]
s[2] = sp[1] + s[0]*l[1]

In [ ]:
d[2] = s[2]*l[1] - s[1]*l[2]
d[2]

In [ ]:
p = simplify(d[2].subs(y,0))
p

In [ ]:
sols = solve(p)
print("All solutions: " + str(sols))
print("Filtered solutions:")
for i in range(len(sols)):
    if re(sols[i]) > 0 and  im(sols[i]) < 0:
        print("w_" + str(i+1) + " = " + str(sols[i]))

### Third iteration

In [ ]:
lp[2] = diff(l[2],y)
sp[2] = diff(s[2],y)
l[3] = lp[2] + s[2] + l[0]*l[2]
s[3] = sp[2] + s[0]*l[2]
d[3] = s[3]*l[2] - s[2]*l[3]
p = simplify(d[3].subs(y,0))
sols = solve(p)
print("All solutions: " + str(sols))
print("Filtered solutions:")
for i in range(len(sols)):
    if re(sols[i]) > 0 and  im(sols[i]) < 0:
        print("w_" + str(i+1) + " = " + str(sols[i]))

### Fourth iteration

In [ ]:
lp[3] = diff(l[3],y)
sp[3] = diff(s[3],y)
l[4] = lp[3] + s[3] + l[0]*l[3]
s[4] = sp[3] + s[0]*l[3]
d[4] = s[4]*l[3] - s[3]*l[4]
p = simplify(d[4].subs(y,0))
sols = solve(p)
print("All solutions: " + str(sols))
print("Filtered solutions:")
for i in range(len(sols)):
    if re(sols[i]) > 0 and  im(sols[i]) < 0:
        print("w_" + str(i+1) + " = " + str(sols[i]))